In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
igfold = 0
os.system('mkdir -p ./data_folds/fold_' + str(igfold))

0

In [2]:
!nvidia-smi

Tue Jan 17 14:11:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:86:00.0 Off |                    0 |
| N/A   34C    P0    42W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:89:00.0 Off |                    0 |
| N/A   

In [3]:
import os

import cudf
import glob

In [4]:
dataset = 'train'
ty = 'clicks'

sub_prev_week  =  ['2022-08-22', '2022-08-29']
sub_prev_2week =  ['2022-08-15', '2022-08-29']
sub_prev_3week =  ['2022-08-08', '2022-08-29']

train_prev_week  = ['2022-08-15', '2022-08-22']
train_prev_2week = ['2022-08-08', '2022-08-22']
train_prev_3week = ['2022-08-01', '2022-08-22']

train4th_file = './data/xgb_train_x.parquet'
sub_file = '../../data/test.parquet'

In [5]:
files = sorted(
    glob.glob('../../data/' + '/train/interim/*.parquet')
)

prefixes = ['p1w', 'p2w', 'p3w']

df = cudf.concat(
    [
        cudf.read_parquet(x) for x in sorted(files)
    ] 
).reset_index(drop=True)

for dataset in ['train', 'sub']:
    out = []
    if dataset == 'train':
        weeks_ints = [train_prev_week, train_prev_2week, train_prev_3week]
        df_tmp = cudf.read_parquet('./data/xgb_train_x.parquet')
    elif dataset == 'sub':
        weeks_ints = [sub_prev_week, sub_prev_2week, sub_prev_3week]
        df_tmp = cudf.read_parquet('../../data/test.parquet')
    else:
        assert 0!=0
    prefix = 'p0w'
    df_tmp['clicks'] = (df_tmp['type']=='clicks').astype('int8')
    df_tmp['carts'] = (df_tmp['type']=='carts').astype('int8')
    df_tmp['orders'] = (df_tmp['type']=='orders').astype('int8')
    df_tmp = df_tmp[['aid','clicks', 'carts', 'orders']].groupby(['aid']).sum().reset_index()
    df_tmp = df_tmp.rename(columns={
        'clicks': prefix+'_clicks',
        'carts': prefix+'_carts',
        'orders': prefix+'_orders',
    })
    out.append(df_tmp)

        
    for i, week_int in enumerate(weeks_ints):
        df_tmp = df[
            (cudf.to_datetime(df['ts']/1000, unit='s')>=cudf.to_datetime(week_int[0]))&
            (cudf.to_datetime(df['ts']/1000, unit='s')<cudf.to_datetime(week_int[1]))
        ]
        prefix = prefixes[i]
        df_tmp['clicks'] = (df_tmp['type']=='clicks').astype('int8')
        df_tmp['carts'] = (df_tmp['type']=='carts').astype('int8')
        df_tmp['orders'] = (df_tmp['type']=='orders').astype('int8')
        df_tmp = df_tmp[['aid','clicks', 'carts', 'orders']].groupby(['aid']).sum().reset_index()
        df_tmp = df_tmp.rename(columns={
            'clicks': prefix+'_clicks',
            'carts': prefix+'_carts',
            'orders': prefix+'_orders',
        })
        out.append(df_tmp)
    
    prefix = 'p0w'
    if dataset == 'train':
        df_tmp = cudf.read_parquet(train4th_file)
    else:
        df_tmp = cudf.read_parquet(sub_file)
    df_tmp['clicks'] = (df_tmp['type']=='clicks').astype('int8')
    df_tmp['carts'] = (df_tmp['type']=='carts').astype('int8')
    df_tmp['orders'] = (df_tmp['type']=='orders').astype('int8')
    df_tmp = df_tmp[['aid','clicks', 'carts', 'orders']].groupby(['aid']).sum().reset_index()
    df_tmp = df_tmp.rename(columns={
        'clicks': prefix+'_clicks',
        'carts': prefix+'_carts',
        'orders': prefix+'_orders',
    })
    out.append(df_tmp)
    
    df_tmp = cudf.concat(out)
    df_tmp = df_tmp.fillna(0)
    df_tmp = df_tmp.groupby(['aid']).sum()
    df_tmp = df_tmp.reset_index()
    df_tmp.to_parquet('./data/' + dataset + '_cand_meta_comb.parquet')

In [6]:
dataset = 'train'
df_tmp = cudf.read_parquet(train4th_file)
df_day_map = cudf.DataFrame({
    'day': [21, 22, 23, 24, 25, 26, 27, 28],
    'day_': [0, 0, 1, 2, 3, 4, 5, 6]
})
df_tmp['day'] = cudf.to_datetime(df_tmp['ts']/1000, unit='s').dt.day
df_tmp = df_tmp.merge(
    df_day_map,
    how='left',
    on=['day']
)
prefix = 'sday'

df_tmp['clicks'] = (df_tmp['type']=='clicks').astype('int8')
df_tmp['carts'] = (df_tmp['type']=='carts').astype('int8')
df_tmp['orders'] = (df_tmp['type']=='orders').astype('int8')
df_tmp = df_tmp[['aid', 'day_', 'clicks', 'carts', 'orders']].groupby(['aid', 'day_']).sum().reset_index()
df_tmp = df_tmp.rename(columns={
    'clicks': prefix+'_clicks',
    'carts': prefix+'_carts',
    'orders': prefix+'_orders',
})
df_tmp.to_parquet('./data/' + dataset + '_cand_day_meta_comb.parquet')

In [7]:
dataset = 'sub'
df_tmp = cudf.read_parquet(sub_file)
df_day_map = cudf.DataFrame({
    'day': [28, 29, 30, 31, 1, 2, 3, 4],
    'day_': [0, 0, 1, 2, 3, 4, 5, 6]
})
df_tmp['day'] = cudf.to_datetime(df_tmp['ts']/1000, unit='s').dt.day
df_tmp = df_tmp.merge(
    df_day_map,
    how='left',
    on=['day']
)
prefix = 'sday'

df_tmp['clicks'] = (df_tmp['type']=='clicks').astype('int8')
df_tmp['carts'] = (df_tmp['type']=='carts').astype('int8')
df_tmp['orders'] = (df_tmp['type']=='orders').astype('int8')
df_tmp = df_tmp[['aid', 'day_', 'clicks', 'carts', 'orders']].groupby(['aid', 'day_']).sum().reset_index()
df_tmp = df_tmp.rename(columns={
    'clicks': prefix+'_clicks',
    'carts': prefix+'_carts',
    'orders': prefix+'_orders',
})
df_tmp.to_parquet('./data/' + dataset + '_cand_day_meta_comb.parquet')

In [9]:
!cp -r ./data/train_cand_meta_comb.parquet ./data_folds/fold_0/
!cp -r ./data/train_cand_meta_comb.parquet ./data_folds/fold_1/
!cp -r ./data/train_cand_meta_comb.parquet ./data_folds/fold_2/
!cp -r ./data/train_cand_meta_comb.parquet ./data_folds/fold_3/
!cp -r ./data/train_cand_meta_comb.parquet ./data_folds/fold_4/

In [10]:
!cp -r ./data/sub_cand_meta_comb.parquet ./data_folds/fold_0/
!cp -r ./data/sub_cand_meta_comb.parquet ./data_folds/fold_1/
!cp -r ./data/sub_cand_meta_comb.parquet ./data_folds/fold_2/
!cp -r ./data/sub_cand_meta_comb.parquet ./data_folds/fold_3/
!cp -r ./data/sub_cand_meta_comb.parquet ./data_folds/fold_4/